# OpenSpecimenAPIconnector.py tutorial

In [1]:
%%capture
# Install a pip package in the current Jupyter kernel

import sys
!{sys.executable} -m pip install OpenSpecimenAPIconnector
!{sys.executable} -m pip install faker
!{sys.executable} -m pip install names

In [2]:
%%capture
#Creating ID-dict for deleting the Tutorial Data
ID = {}

In [3]:
%%capture
#import the packages which are needed
import json
import pandas
import ipywidgets as widgets
import faker
from IPython.display import display

### Importing the PIP-package as:

In [4]:
import OpenSpecimenAPIconnector as OSconn
import OpenSpecimenAPIconnector.os_core as os_core
import OpenSpecimenAPIconnector.os_util as os_util

In [5]:
# Setting the URL of the OpenSpecimen distribution
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
input_=input()
if input_ == '':
    base_url = 'http://localhost:9000'
else:
    base_url = input_
base_url +='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter

The whole URL for API Calls is: http://localhost:9000/openspecimen/rest/ng


In [6]:
#Setting the login data, in particular (loginname, passwort)
loginname='admin';
password='Login@123'
print('Loginname(admin):')
input_=input()
if input_ != '':
    loginname=input_
print('Password(Login@123):')
input_=input()
if input_ != '':
    password=input_
auth = (loginname, password)
print('Your (loginname,password) is (' + str(loginname) + ',' +str(password) + ')')

Loginname(admin):

Password(Login@123):

Your (loginname,password) is (admin,Login@123)


### Configuring the package with the Logindata
If the logindata is wrong the next cell will throw an error, else everything is fine.

In [7]:
OSconn.config_manager.set_login(url = base_url, auth = auth)

## Creating a Specimen
OpenSpecimen has an internal hierarchy. Precisely it is,
**Institute>Site>Collection Protocol>Participant>Event>Visit>Specimen**.
It is not possible to create a object from a lower instance without all the higher, except visit and specimen in on go. It is not possible for example to create a Site without an Institute.

### Institutes


In [8]:
# initialize the institutes class
institutes_util = os_util.institutes_util()
response = institutes_util.create_institute(institutename = 'Demo Institute')
ID['institute'] = response['id']
print(response)

{'id': 2, 'name': 'Demo Institute', 'activityStatus': 'Active', 'usersCount': 0}


#### Creating with the Core-package.
One can also make the calls via the os_core package. This is more advanced, because the user has to know all the paramteres which can and have to be passed to OpenSpecimen. The advantage of this method is, if the OpenSpecimen distribution has custom forms, it is still useable without further programming. Else the new keywords have to be added to the classes.

In the next cell it is shown how the core package can be used.

In [9]:
institutes = os_core.institutes()
params = '{\"name\":\"Demo Institute2\"}'
response = institutes.create_institute(params = params)
ID['institute2'] = response['id']
print(response)

{'id': 3, 'name': 'Demo Institute2', 'activityStatus': 'Active', 'usersCount': 0}


In [10]:
#The response should be Institute with same name already exists.
params = '{\"name\":\"Demo Institute\"}'
response = institutes.create_institute(params = params)
print(response)

[{'code': 'INSTITUTE_DUP_NAME', 'message': 'Institute with same name already exists.'}]


### Site

In [11]:
#Initialize sites
site_util = os_util.site_util()
response = site_util.create_sites(name = 'Demo Site', institutename = ' Demo Institute', type_ = 'not specified')
ID['site']=response['id']
print(response)

{'opComments': None, 'id': 1, 'name': 'Demo Site', 'instituteName': 'Demo Institute', 'code': None, 'type': 'Not Specified', 'activityStatus': 'Active', 'cpCount': 0, 'coordinators': [], 'address': None, 'extensionDetail': None}


### Collection Protocol

In [12]:
#initialize
collection_protocol_util = os_util.collection_protocol_util()
response = collection_protocol_util.create_cp(short_title = 'Demo CP', title = 'Demo Collection Protocol', pi_mail = 'admin', sites = ['Demo Site'])
ID['cp'] = response['id']
print(response)


{'id': 1, 'shortTitle': 'Demo CP', 'title': 'Demo Collection Protocol', 'principalInvestigator': {'id': 2, 'type': 'SUPER', 'firstName': 'System', 'lastName': 'Administrator', 'loginName': 'admin', 'domain': 'openspecimen', 'emailAddress': 'admin@localhost', 'instituteName': 'Biobank Institute', 'primarySite': None, 'admin': True, 'instituteAdmin': False, 'manageForms': False, 'cpCount': 0, 'creationDate': None, 'activityStatus': 'Active'}, 'manualPpidEnabled': False, 'specimenCentric': False, 'coordinators': [], 'cpSites': [{'id': 1, 'siteName': 'Demo Site', 'code': None}], 'consentsWaived': False, 'manualVisitNameEnabled': False, 'manualSpecLabelEnabled': False, 'bulkPartRegEnabled': False, 'barcodingEnabled': False, 'closeParentSpecimens': False, 'visitCollectionMode': 'ALL_SPMNS', 'visitNamePrintMode': 'NONE', 'spmnLabelPrePrintMode': 'NONE', 'spmnLabelPrintSettings': [], 'distributionProtocols': [], 'activityStatus': 'Active', 'aliquotLabelFmtToUse': '%PSPEC_LABEL%_%PSPEC_UID%'}


### Participant Registration

In [13]:
#initialize
cpr_util = os_util.cpr_util()

response = cpr_util.create_registration(regdate = '2021-03-17', cpid = ID['cp'], ppid = "Demo Participant",lastname = "Sepp")
ID['cpr'] = response['id'] #is needed afterwards
ID['participant'] = response['participant']['id']
print(response)


{'opComments': None, 'id': 1, 'participant': {'id': 1, 'source': 'OpenSpecimen', 'lastName': 'Sepp', 'races': [], 'pmis': [], 'ethnicities': [], 'activityStatus': 'Active', 'phiAccess': True, 'registeredCps': [], 'cpId': -1, 'reqRegInfo': False, 'forceDelete': False}, 'cpId': 1, 'cpTitle': 'Demo Collection Protocol', 'cpShortTitle': 'Demo CP', 'ppid': 'IntegrationTestPPID', 'barcode': None, 'activityStatus': 'Active', 'registrationDate': 1615939200000, 'externalSubjectId': None, 'site': None, 'consentDetails': None, 'specimenLabelFmt': None, 'aliquotLabelFmt': '%PSPEC_LABEL%_%PSPEC_UID%', 'derivativeLabelFmt': None, 'forceDelete': False}


### Event
There are two different ways starting from here for creating a Specimens. The first one is to create an **event**, then a **visit** and last but not least the **specimen**. The other way is to create a **specimen** and a **visit** in one go. the first one, is  prefferable if the events are known before, beause it is easy to integrate in the workflow via the parameter **code** of the event.

In [14]:
#initialize
cpevent_util = os_util.cpevent_util()

response = cpevent_util.create_event(label = 'Demo Label', point = 0, cp = 'Demo Collection Protocol', 
                                     site = 'Demo Site', diagnosis = 'Not Specified', status = 'Not Specified',
                                     activity = 'Active', unit = 'DAYS', code ='DemoEventCode')
ID['event'] = response['id']
print(response)

{'id': 1, 'eventLabel': 'Demo Label', 'eventPoint': 0, 'eventPointUnit': 'DAYS', 'cpId': 1, 'collectionProtocol': 'Demo Collection Protocol', 'cpShortTitle': 'Demo CP', 'defaultSite': 'Demo Site', 'clinicalDiagnosis': 'Not Specified', 'clinicalStatus': 'Not Specified', 'activityStatus': 'Active', 'offset': 0, 'offsetUnit': 'DAYS'}


### Visit

In [15]:
#initialize
visit_util = os_util.visit_util()

response = visit_util.add_visit(name = 'Demo Visit', site = 'Demo Site', cprid = ID['cpr'])
ID['visit'] = response['id']
print('response')

response


### Specimen

In [16]:
#initialize
specimen_util = os_util.specimen_util()

response = specimen_util.create_specimen(specimenclass = 'Fluid', specimentype = 'Bile', pathology = 'Malignant', anatomic = 'Anal Canal',  
                 laterality = 'Left', initqty = 10, avaqty =10, visitid = ID['visit'], recqlt = 'Acceptable', userid = 2,
                 label = 'Demo Specimen Label', colltime = '2021-03-01', rectime = '2021-03-01')
ID['specimen'] = response['id']
print(response)

{'opComments': None, 'id': 1, 'cpId': 1, 'cprId': 1, 'ppid': 'IntegrationTestPPID', 'eventId': None, 'eventCode': None, 'eventLabel': None, 'visitId': 1, 'visitName': 'Demo Visit', 'visitStatus': 'Complete', 'sprNo': None, 'visitDate': 1616420073000, 'cpShortTitle': 'Demo CP', 'reqId': None, 'sortOrder': None, 'label': 'Demo Specimen Label', 'barcode': None, 'type': 'Bile', 'specimenClass': 'Fluid', 'lineage': 'New', 'anatomicSite': 'Anal canal', 'laterality': 'Left', 'status': 'Collected', 'reqLabel': None, 'pathology': 'Malignant', 'initialQty': 10, 'availableQty': 10, 'concentration': None, 'parentId': None, 'parentLabel': None, 'storageLocation': {'id': -1, 'name': None, 'mode': None, 'positionX': None, 'positionY': None, 'position': None, 'reservationId': None}, 'storageType': None, 'collectionContainer': None, 'collectionDate': None, 'storageSite': None, 'activityStatus': 'Active', 'createdOn': 1614556800000, 'createdBy': None, 'code': None, 'distributionStatus': None, 'freezeTha

### Visit and Specimen in one go

In [17]:
response = visit_util.add_visit_speci(name = 'Demo Visit 2', lineage = 'New', av_qty = 10, user = 2, init_qty =10,
                                      spec_class = 'Fluid', spec_type = 'Bile', anat_site = 'Anal Canal', 
                                      path = 'Malignant', site = 'Demo Site', speclabel = 'Demo SP Label',
                                      cpid =ID['cp'], ppid = 'Demo PPID2', cprid = ID['cpr'], 
                                      colltime = '2021-03-01', rectime = '2021-03-01')
ID['visit2']=response['visit']['id']
ID['specimen2']=response['specimens'][0]['id']
print(response)

{'visit': {'id': 2, 'cpId': 1, 'name': 'Demo Visit 2', 'status': 'Complete', 'visitDate': 1616420073190, 'totalPendingSpmns': 0, 'pendingPrimarySpmns': 0, 'plannedPrimarySpmnsColl': 0, 'uncollectedPrimarySpmns': 0, 'unplannedPrimarySpmnsColl': 0, 'storedSpecimens': 0, 'notStoredSpecimens': 0, 'distributedSpecimens': 0, 'closedSpecimens': 0, 'cprId': 1, 'ppid': 'IntegrationTestPPID', 'cpTitle': 'Demo Collection Protocol', 'cpShortTitle': 'Demo CP', 'clinicalDiagnoses': [], 'diagnosisList': [], 'activityStatus': 'Active', 'site': 'Demo Site', 'sprLocked': False, 'forceDelete': False, 'description': 'Unplanned Visit / Demo Visit 2'}, 'specimens': [{'opComments': None, 'id': 2, 'cpId': 1, 'cprId': 1, 'ppid': 'IntegrationTestPPID', 'eventId': None, 'eventCode': None, 'eventLabel': None, 'visitId': 2, 'visitName': 'Demo Visit 2', 'visitStatus': 'Complete', 'sprNo': None, 'visitDate': 1616420073190, 'cpShortTitle': 'Demo CP', 'reqId': None, 'sortOrder': None, 'label': 'Demo SP Label', 'barcod

 ## Removing the Tutorial Data

In [23]:
#Ask if the user wants to remove the data
print('Press enter to remove the Tutorial Data. If you like to keep it and delete it manually write anything.')
input_ =input()
if input_ =='':
    specimen_util.delete_specimens([ID['specimen'],ID['specimen2']])
    visit = os_core.visit()
    visit.delete_visit(ID['visit'])
    visit.delete_visit(ID['visit'])
    event = os_core.collection_protocol_event()
    event.delete_event(ID['event'])
    cpr = os_core.collection_protocol_registration()
    cpr.delete_participant(ID['cpr'])
    cp = os_core.collection_protocol()
    cp.delete_collection_protocol(ID['cp'])
    site = os_core.sites()
    site.delete_sites(ID['site'])
    institutes.delete_institute(ID['institute'])

Press enter to remove the Tutorial Data. If you like to keep it and delete it manually write anything.

